In [1]:
import os
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
from gslTranslater.utils.common import read_yaml, create_directories
from gslTranslater.constants import *

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Dev\\Upwork\\GSL\\GSL-Project'

In [4]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    cnn_lstm_model_path: Path
    params_image_size: list
    params_weights: str
    params_classes: int

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(prepare_base_model.root_dir),
            cnn_lstm_model_path=Path(prepare_base_model.cnn_lstm_model_path),
            params_image_size=params.IMAGE_SIZE,
            params_weights=params.WEIGHTS,
            params_classes=params.CLASSES
        )

        return prepare_base_model_config

In [6]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def prepare_model(self):
        # Load the pretrained ResNet50 model without the top layer
        cnn_base = tf.keras.applications.ResNet50(
            weights=self.config.params_weights,
            include_top=False,
            input_shape=tuple(self.config.params_image_size)
        )

        # Freeze the CNN layers
        for layer in cnn_base.layers:
            layer.trainable = False

        # Define the full model architecture
        model = tf.keras.Sequential()

        # TimeDistributed layer applies the CNN to each frame independently
        model.add(tf.keras.layers.TimeDistributed(cnn_base, input_shape=(None, *self.config.params_image_size)))
        model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))

        # Add Masking layer to handle padded frames
        model.add(tf.keras.layers.Masking(mask_value=0.0))

        # LSTM layer processes the sequence of frame features
        model.add(tf.keras.layers.LSTM(self.config.params_classes, return_sequences=False))

        # Save the model
        model.save(self.config.cnn_lstm_model_path)
        print(f"CNN-LSTM base model saved successfully at {self.config.cnn_lstm_model_path}")


In [7]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.prepare_model()

except Exception as e:
    raise e

[2024-08-28 22:12:46,403: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 22:12:46,406: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 22:12:46,408: INFO: common: created directory at: artifacts]
[2024-08-28 22:12:49,181: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-08-28 22:13:09,714: WARNING: save: Found untraced functions such as lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 55). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: artifacts\prepare_base_model\cnn_lstm_model.pth\assets
[2024-08-28 22:13:17,154: INFO: builder_impl: Assets written to: artifacts\prepare_base_model\cnn_lstm_model.pth\assets]
CNN-LS